In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D

from glob import glob
import time
import os

from sklearn.svm import LinearSVC 
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

import pipeline as pl

# Reload custum module everytime
%load_ext autoreload
%aimport pipeline
%autoreload 1


# Load Training and Test Datasets

In [ ]:
def get_image_files(path):
    image_folders     = os.listdir(path)
    image_files = {key: [] for key in image_folders}
    for folder in image_folders:

        subfolders = os.listdir(path+folder)
        print("Subfolders of {}:".format(folder))    
        print(subfolders)

        for subfolder in subfolders:
            abs_path = path + folder + '/' + subfolder + '/*.png'
            if not subfolder.startswith('.'):
                files =  glob(abs_path)
                [image_files[folder].append(file) for file in files]
                
    dataset = {'files': [], 'labels': []}
    for key in image_files:
        print(key)
        for file in image_files[key]:
            dataset['files'].append(file)
            if key == 'vehicles':
                dataset['labels'].append(1)
            elif key == 'non-vehicles':
                dataset['labels'].append(0)
                
    np.random.seed(42)
    np.random.shuffle(dataset['files'])
    np.random.seed(42)
    np.random.shuffle(dataset['labels'])            
    
    return dataset
dataset = get_image_files(mldata_path)
print(len(dataset['files']))
print(len(dataset['labels']))

In [ ]:
for n in range(0, 3):
    plt.imshow(mpimg.imread(dataset['files'][n]))
    print(dataset['labels'][n])
    plt.show()

In [ ]:
mldata_path = '/home/timo/Documents/mldata/project_5/'

image_files = pl.get_image_files(path=mldata_path)

number_of_images = {key: len(image_files[key]) for key in image_files}
for key in number_of_images:
    print(number_of_images[key])
    

In [ ]:
img = mpimg.imread(image_files['vehicles'][0])

colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

hog = pl.get_hog_features(gray, orient, pix_per_cell, cell_per_block, vis=True)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 3))
axes[0].imshow(img)
axes[1].imshow(hog[1], 'gray')
axes[2].plot(hog[0])
axes[3].hist(hog[0], bins=32, range=[0.025,0.2])
plt.show()

In [ ]:
features = {key: [] for key in image_files}
labels   = {key: [] for key in image_files}
nb_samples = 100# min(len(image_files['vehicles']), len(image_files['non-vehicles']))


for key in image_files:
    features[key] = pl.extract_features(image_files[key][0:nb_samples], spatial_feat=False)
    
labels['vehicles']      = np.ones((features['vehicles'].shape[0],1))
labels['non-vehicles']  = np.zeros((features['non-vehicles'].shape[0],1))

#print("\nShape of features and labels:")
#print(features['vehicles'].shape)
#print(labels['vehicles'].shape)

In [ ]:
for key in features:
    plt.figure(figsize=(20,3))
    plt.hist(features[key].ravel(), bins=256)
    plt.grid('on')
    plt.show()

In [ ]:
X, y          = pl.get_shuffle_split_train_test_datasets(features=features, nb_samples=nb_samples)
images, files = pl.get_shuffle_split_train_test_images(image_files, nb_samples=nb_samples)

print("Number of samples \ntraining:\t{0} \ntest:\t\t{1}".format(X['train'].shape[0], X['test'].shape[0]))

In [ ]:
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()

svc.fit(X['train'], y['train'])
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X['test'], y['test']), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
n_predict = 10
print('My SVC predicts: ', svc.predict(X['test'][0:n_predict]))
print('For these',n_predict, 'labels: ', y['test'][0:n_predict].T)
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

In [ ]:
n = 6
plt.imshow(images['test'][n])
print(y['test'][n])
plt.show()

In [ ]:
n = 0
img = mpimg.imread(image_files['vehicles'][n])
plt.imshow(img)
print(labels['vehicles'][n])
plt.show()

In [ ]:
np.random.seed(42)
np.random.shuffle(image_files['vehicles'])
np.random.seed(42)
np.random.shuffle(labels['vehicles'])